<a href="https://colab.research.google.com/github/rebeccachery/haitian-translator-audiosplitter/blob/main/AudioSilenceSpliter_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yt-dlp pydub ffmpeg openai-whisper numpy librosa openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 12.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.6 MB/s eta 0:00:00
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=0f5a2fc22956047161c41965262e5196dec0618d628035d044a438282b0741dd
  Stored in directory: /root/.cache/pip/wheels/26/21/0c/c26e09dff860a9071683e279445262346e008a9a1d2142c4ad
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=87e79623d8fa04bca9b9580293130c8bd1b61100d082c44bef1db2526feb7bc9
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built ffmpeg openai-whisper


In [ ]:
import yt_dlp
import numpy as np
import librosa
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence
import json
from google.colab import userdata
from openai import OpenAI
from urllib.parse import urlparse, parse_qs


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [ ]:
client = OpenAI(api_key=userdata.get('OPEN_AI'))

def extract_youtube_id(url):
    parsed_url = urlparse(url)
    if parsed_url.netloc in ["www.youtube.com", "youtube.com"]:
        query_params = parse_qs(parsed_url.query)
        return query_params.get("v", [None])[0]
    elif parsed_url.netloc in ["youtu.be"]:  # Handle shortened YouTube URLs
        return parsed_url.path.lstrip("/")
    return None

def download_audio(youtube_url, output_path="audio.mp3"):
    """Download audio from a YouTube video."""

    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": output_path,
        "postprocessors": [{
            "key": "FFmpegExtractAudio",
            "preferredcodec": "mp3",
            "preferredquality": "192"
        }],
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

    print(f"Downloaded audio saved as {output_path}")

def convert_to_wav(input_path="audio.mp3", output_path="audio.wav"):
    """Convert MP3 to WAV with 16kHz sample rate."""
    audio = AudioSegment.from_file(input_path)
    audio = audio.set_frame_rate(16000).set_channels(1)
    audio.export(output_path, format="wav")
    print(f"Converted to WAV: {output_path}")
    return output_path

def split_audio_on_silence(audio_path,name, min_silence_len=800, silence_thresh=-40):
    """Splits audio into smaller chunks based on silence."""
    audio = AudioSegment.from_wav(audio_path)

    chunks = split_on_silence(audio,
                              min_silence_len=min_silence_len,  # Silence duration (in ms)
                              silence_thresh=silence_thresh,    # Silence threshold in dB
                              keep_silence=400)                 # Keep some silence for context

    chunk_paths = []
    for i, chunk in enumerate(chunks):
        chunk_path = f"{name}/chunk_{i}.wav"
        chunk.export(chunk_path, format="wav")
        chunk_paths.append(chunk_path)

    print(f"Split audio into {len(chunks)} chunks.")
    return chunk_paths

def transcribe_chunks(chunk_paths,name, model_size="small"):
    """Transcribes each audio chunk using Whisper."""
    # model = whisper.load_model(model_size)


    transcript_data = []

    for i, chunk_path in enumerate(chunk_paths):
        print(f"Transcribing {chunk_path}...")
        audio, sr = librosa.load(chunk_path, sr=16000)  # Load with 16kHz sample rate
        with open(chunk_path, "rb") as audio_file:
          result = client.audio.transcriptions.create(
          model="gpt-4o-transcribe",
          file=audio_file # Pass the file-like object
          )

        transcript_data.append({
            "chunk_id": i,
            "file": chunk_path,
            "transcription": result.text
        })

    return transcript_data

def save_transcription(transcript_data, output_file="transcription.json"):
    """Saves transcription data as a JSON file."""
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(transcript_data, f, indent=4, ensure_ascii=False)
    print(f"Transcription saved to {output_file}")


SecretNotFoundError: Secret OPEN_AI does not exist.

In [ ]:
  # Example Usage
video_url = "https://www.youtube.com/watch?v=S2ejTQtXdZM&t=21s"

"""Complete pipeline: Download, split, transcribe, and save results."""

download_audio(video_url, "audio")

name = extract_youtube_id(video_url)

os.makedirs(name, exist_ok=True)


In [ ]:
wav_path = convert_to_wav("audio.mp3", "audio.wav")
chunk_paths = split_audio_on_silence(wav_path,name)


In [ ]:
transcript_data = transcribe_chunks(chunk_paths,name)
save_transcription(transcript_data,f"{name}/transcription.json")

In [ ]:
audio = AudioSegment.from_file(f"{name}/chunk_4.wav", format="wav")
audio